# Qual a média de nota por matéria e trimestre?

In [99]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, MetaData, Table, inspect 

In [100]:
engine = create_engine('sqlite:///:memory:') # Criando banco SQL
tabela_notas = pd.read_excel('files/tabela_nota.xlsx') # Importando tabela de notas
tabela_notas.drop(tabela_notas.loc[tabela_notas['id_materia']==0].index, inplace=True) # Não é possivel saber qual materia ele fez
tabela_notas['id'] = range(tabela_notas.shape[0])
tabela_notas.set_index('id', inplace=True)
tabela_notas.index = tabela_notas.index + 1
tabela_notas.to_sql('notas', engine) 
inspector = inspect(engine) 
print(inspector.get_table_names())

['notas']


## Média de notas por matéria

In [101]:
# Funcao que busca a tabela de notas da materia especificada para calcular a média
def busca_tabela_notas_materia(id_materia: int):
    query = f'select * from notas where id_materia = {id_materia}'
    tabela_notas_materia = pd.read_sql(query, engine)
    return tabela_notas_materia

In [102]:
# Funcao para calcular media das notas de cada materia
def media_de_nota_por_materia(id_materia: int) -> float:
    tabela_notas_materia = busca_tabela_notas_materia(id_materia)
    media_notas = tabela_notas_materia['nota'].mean()
    return media_notas

In [103]:
# Exportar no excel
def exporta_para_excel(itens: list, nome_coluna_itens: str, valores: list, nome_coluna_valores: str, caminho: str):
    dados = pd.DataFrame({nome_coluna_itens: itens, nome_coluna_valores: valores})
    dados.to_excel(caminho, index=False)

In [104]:
# busca lista de todas as materias
materias = pd.read_excel('files/tabela_materia.xlsx')['descricao']
materias = list(materias)

In [112]:
medias_notas_por_materias = []
for i in range(1, len(materias) + 1):
    medias_notas_por_materias.append(media_de_nota_por_materia(i))
    
# Exportando para excel um dataframe com os valores das médias das notas
exporta_para_excel(materias, 'Materias', medias_notas_por_materias, 'Médias das Notas', 'files/files_questions/question_1/media_por_materia.xlsx')

## Média de nota das matérias por trimestre

In [106]:
# Funcao que busca a tabela de notas do trimestre especificada para calcular a média
def busca_tabela_notas_trimestre(trimestre: int):
    selecao = tabela_notas['data_inicio_trimestre'] == trimestres[trimestre]
    return tabela_notas[selecao]

In [107]:
# Funcao para calcular media das notas das materias em cada trimestre
def media_de_nota_por_trimestre(trimestre: int) -> float:
    tabela_notas = busca_tabela_notas_trimestre(trimestre)
    media_notas = tabela_notas['nota'].mean()
    return media_notas

In [108]:
# Funcao que pega todos os trimestres de um ano especifico
def pega_trimestres(trimestre: int) -> str:
    tabela_trimestres = busca_tabela_notas_trimestre(trimestre)
    data_trimestre = tabela_trimestres['data_inicio_trimestre'].tolist() 
    return data_trimestre

In [110]:
trimestres = list(tabela_notas['data_inicio_trimestre'].drop_duplicates())
trimestres = sorted(trimestres)

In [115]:
anos = ['2017', '2018', '2019', '2020', '2021'] # todos os anos presente na lista
iterr_list = [0, 1, 2, 3, 5] # Conseguir percorrer corretamente a lista de trimestres

# Exportando os valores das medias por trimestre de cada ano, em arquivo excel separado
for j, k in zip(iterr_list, anos):
    list_trimestres = []
    list_media_notas = []
    for i in range(j, len(trimestres), 5):
        list_media_notas.append(media_de_nota_por_trimestre(i))
        list_trimestres.append(pega_trimestres(i)[0])
   
    exporta_para_excel(list_trimestres, 'Trimestre', list_media_notas, 'Média das Notas', f'files/files_questions/question_1/media_por_trimestre-{k}.xlsx')